# 整体概览
1. 数据加载

   即数据读取

2. 数据探索

   通过info()查看数据类型，主要是非数值类型的字段需要进行特殊处理。了解到非数值类型有 `lan`, `os`, `osv`, `version`, `fea_hash` 共5个

   查看数据缺失值，发现`lan`和`osv`字段有缺失值，需要进行填充

   查看数据唯一值个数，发现`os`字段唯一值仅2，表明此字段并不具有很强的区分性，删除；`sid`字段是样本的`id`后续处理也暂不考虑

3. 特征工程

   + 处理4个非数值类型（`os`删除，所以是4个），即对非数值类型进行转换，使其为数值类型
     + `lan`是设备采用的语言，认为结果（样本`sid`是否会作弊）也是可能与地域有关的，所以将地域相近的编码挨着；又由于此字段有缺失值，因此用众数对缺失值进行填充
     + `osv`操作系统版本，有缺失值用众数填充；对于版本信息仅保留数字，过长的进行截断，详见代码注释
     + `version`是`app`版本，结合生活经验`V3`很可能是某个 `app` 的第3版，因此，转换为数值3，依次类推
     + `fea_hash` 用户特征编码，`fea_hash1`也是用户特征编码，两个一起处理，由于物理意义不明，这里直接按照长度进行处理，默认超过16的意义不大，

   + 增加新的特征
     + 构造时间相关特征，主要有`year,month,day,week_day,hour,minute,diff_time（时间差）`
     + 构造`dev`相关特征，主要有设备的宽高比`dev_ratio`，设备的面积`dev_area`
     + 构造`app`版本与操作系统版本相关特征，主要是二者作差。可能就是说，会不会手机`app`版本低而操作系统版本高会比较容易作弊，这样子的理解

4. 建模 训练 预测

   使用XGBoost模型，其原理网上资源有很多，就不做过多介绍了

   关于调参，也是在经验参数上试了好几次，哪些参数比较好这样子，最终的参数见代码

5. 很感谢百度飞浆给的平台，免费GPU资源很可~

6. 需要说明的是，同代码，XGBoost模型使用CPU最终成绩为89.074，使用GPU最终成绩为88.98。个人猜测，或许是实现差异吧

In [1]:
from paddle.io import Dataset
from paddle import min
from datetime import datetime
import pandas as pd
import warnings

warnings.filterwarnings('ignore')  # 警告信息较多，过滤掉警告信息

# Step1: 数据加载

In [2]:
train = pd.read_csv('./data/data100910/train.csv')
test = pd.read_csv('./data/data100910/test1.csv')
train

,Unnamed: 0,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,label,lan,media_id,...,os,osv,package,sid,timestamp,version,fea_hash,location,fea1_hash,cus_type
0,0,316361,1199,46000.0,0.0,0.0,0.0,1,NaN,104,...,android,9,18,1438873,1.559893e+12,8,2135019403,0,2329670524,601
1,1,135939,893,0.0,0.0,0.0,0.0,1,NaN,19,...,android,8.1,0,1185582,1.559994e+12,4,2782306428,1,2864801071,1000
2,2,399254,821,0.0,760.0,0.0,360.0,1,NaN,559,...,android,8.1.0,0,1555716,1.559837e+12,0,1392806005,2,628911675,696
3,3,68983,1004,46000.0,2214.0,0.0,1080.0,0,NaN,129,...,android,8.1.0,0,1093419,1.560042e+12,0,3562553457,3,1283809327,753
4,4,288999,1076,46000.0,2280.0,0.0,1080.0,1,zh-CN,64,...,android,8.0.0,0,1400089,1.559867e+12,5,2364522023,4,1510695983,582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,499995,392477,1028,46000.0,1920.0,3.0,1080.0,1,zh-CN,144,...,Android,7.1.2,25,1546078,1.559834e+12,7,861755946,79,140647032,373
499996,499996,346134,1001,0.0,1424.0,0.0,720.0,0,NaN,29,...,android,8.1.0,0,1480612,1.559814e+12,3,1714444511,23,2745131047,525
499997,499997,499635,761,46000.0,1280.0,0.0,720.0,0,NaN,54,...,android,6.0.1,9,1698442,1.559676e+12,0,3843262581,25,1326115882,810
499998,499998,239786,917,46001.0,960.0,0.0,540.0,0,zh_CN,109,...,android,5.1.1,0,1331155,1.559840e+12,0,1984296118,225,1446741112,772


In [3]:
# 从上述结果中可以发现，Unnamed:0此列无用 多余 所以去掉
test = test.iloc[:, 1:]
train = train.iloc[:, 1:]
train

,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,label,lan,media_id,ntt,os,osv,package,sid,timestamp,version,fea_hash,location,fea1_hash,cus_type
0,316361,1199,46000.0,0.0,0.0,0.0,1,NaN,104,6.0,android,9,18,1438873,1.559893e+12,8,2135019403,0,2329670524,601
1,135939,893,0.0,0.0,0.0,0.0,1,NaN,19,6.0,android,8.1,0,1185582,1.559994e+12,4,2782306428,1,2864801071,1000
2,399254,821,0.0,760.0,0.0,360.0,1,NaN,559,0.0,android,8.1.0,0,1555716,1.559837e+12,0,1392806005,2,628911675,696
3,68983,1004,46000.0,2214.0,0.0,1080.0,0,NaN,129,2.0,android,8.1.0,0,1093419,1.560042e+12,0,3562553457,3,1283809327,753
4,288999,1076,46000.0,2280.0,0.0,1080.0,1,zh-CN,64,2.0,android,8.0.0,0,1400089,1.559867e+12,5,2364522023,4,1510695983,582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,392477,1028,46000.0,1920.0,3.0,1080.0,1,zh-CN,144,6.0,Android,7.1.2,25,1546078,1.559834e+12,7,861755946,79,140647032,373
499996,346134,1001,0.0,1424.0,0.0,720.0,0,NaN,29,2.0,android,8.1.0,0,1480612,1.559814e+12,3,1714444511,23,2745131047,525
499997,499635,761,46000.0,1280.0,0.0,720.0,0,NaN,54,6.0,android,6.0.1,9,1698442,1.559676e+12,0,3843262581,25,1326115882,810
499998,239786,917,46001.0,960.0,0.0,540.0,0,zh_CN,109,2.0,android,5.1.1,0,1331155,1.559840e+12,0,1984296118,225,1446741112,772


# Step2: 数据探索

In [4]:
# 查看数据集概览
train.info()
# 从结果可以看出大多数都是数值类型
# 非数值类型，即object的有 lan, os, osv, version, fea_hash 5个
# 需要对object类型进行处理，再建模

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 20 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   android_id  500000 non-null  int64  
 1   apptype     500000 non-null  int64  
 2   carrier     500000 non-null  float64
 3   dev_height  500000 non-null  float64
 4   dev_ppi     500000 non-null  float64
 5   dev_width   500000 non-null  float64
 6   label       500000 non-null  int64  
 7   lan         316720 non-null  object 
 8   media_id    500000 non-null  int64  
 9   ntt         500000 non-null  float64
 10  os          500000 non-null  object 
 11  osv         493439 non-null  object 
 12  package     500000 non-null  int64  
 13  sid         500000 non-null  int64  
 14  timestamp   500000 non-null  float64
 15  version     500000 non-null  object 
 16  fea_hash    500000 non-null  object 
 17  location    500000 non-null  int64  
 18  fea1_hash   500000 non-null  int64  
 19  cu

In [5]:
# 查看缺失值
train.isnull().sum()
# 从结果可以看出，lan和osv字段有缺失值

android_id         0
apptype            0
carrier            0
dev_height         0
dev_ppi            0
dev_width          0
label              0
lan           183280
media_id           0
ntt                0
os                 0
osv             6561
package            0
sid                0
timestamp          0
version            0
fea_hash           0
location           0
fea1_hash          0
cus_type           0
dtype: int64

In [6]:
# 查看训练数据唯一值的个数
features = train.columns.tolist()
for feature in features:
    print(feature, train[feature].nunique())  # 打印feature及其数据量的个数
# 由结果可以看出os的唯一值仅2，表明此特征的区分度小，不需要此特征
# sid字段是样本的id，也不需要

android_id 362258
apptype 89
carrier 5
dev_height 798
dev_ppi 92
dev_width 346
label 2
lan 21
media_id 284
ntt 8
os 2
osv 154
package 1950
sid 500000
timestamp 500000
version 22
fea_hash 402980
location 332
fea1_hash 4959
cus_type 58


# Step3：特征工程

## Step3.1: 处理4个object类型的数据（os删除 剩4个）

In [7]:
# 3.1.1 处理第一个object类型的特征：lan  设备采用的语言
train['lan'].value_counts().index

Index(['zh-CN', 'zh', 'cn', 'zh_CN', 'Zh-CN', 'zh-cn', 'ZH', 'CN', 'tw', 'en',
       'zh_CN_#Hans', 'ko', 'zh-TW', 'en-US', 'zh-HK', 'en-GB', 'it', 'TW',
       'ja', 'mi', 'zh-MO'],
      dtype='object')

In [8]:
# 大概是可以看出: 
# zh-CN, zh, cn, zh_CN, Zh-CN, zh-cn, ZH, CN, zh_CN_#Hans 估摸是表示中文
# zh-HK 可能是香港
# tw, zh-TW, TW 可能是表示台湾
# ...
# 可以发现lan字段表示设备采用语言，结合生活常识吧，结果也有可能是和地域有关的，所以处理此字段的时候将其进行编码的时候将这些地域相近的挨着编码
lan_map = {'zh-CN': 1, 'zh': 2, 'cn': 3, 'zh_CN': 4, 'Zh-CN': 5, 'zh-cn': 6, 'ZH': 7, 'CN': 8, 'zh_CN_#Hans': 9,\
'zh-HK': 10, 'zh-MO': 11, 'tw': 12, 'zh-TW':13, 'TW': 14,\
'en': 15, 'en-GB': 16, 'en-US': 17, 'ko': 18, 'it': 19, 'ja': 20, 'mi': 21}
train['lan'] = train['lan'].map(lan_map)
test['lan'] = test['lan'].map(lan_map)
test['lan'].value_counts()

1.0     73046
2.0      9094
3.0      6136
4.0      3391
5.0      2430
6.0       165
7.0        60
8.0        46
12.0       28
15.0       26
9.0        11
18.0        6
13.0        5
17.0        2
10.0        2
16.0        1
14.0        1
Name: lan, dtype: int64

In [9]:
# 由上知lan有缺失值，用众数填充
train['lan'].fillna(1, inplace=True)
test['lan'].fillna(1, inplace=True)

In [10]:
train['lan']

0         1.0
1         1.0
2         1.0
3         1.0
4         1.0
         ... 
499995    1.0
499996    1.0
499997    1.0
499998    4.0
499999    1.0
Name: lan, Length: 500000, dtype: float64

In [11]:
# 3.1.2 处理第二个object类型的特征：osv  操作系统版本
train['osv'].value_counts()
train['osv'].value_counts().index

Index(['8.1.0', '9', '6.0.1', '7.1.1', '9.0.0', '6.0', '5.1', '8.1', '5.1.1',
       '8.0.0',
       ...
       '5.0.3', '10.3.3', '6.0 十核2.0G_HD', '9.1.0', '2.0.1', '5.1.1_22', '4.0',
       '2.3.7', '5.3.0', 'Android_5.0.1'],
      dtype='object', length=154)

In [12]:
# 由上可知，osv有缺失值，用众数填充
train['osv'].fillna('8.1.0', inplace=True)
test['osv'].fillna('8.1.0', inplace=True)

In [13]:
# 可以发现，osv操作系统版本，基本上都是几点几点几这样形式的，还有些有英文字母啥的，这些都需要去掉，才能建模
# 还有一部分系统版本，是有空格或者-的，默认后面这些都不太重要，删了（字符串截取到这里）
def osv_processing(osversion):
    osversion = str(osversion).replace('Android_', '').replace('Android ','').replace('W','')
    if str(osversion).find('.')>0:
        idx1 = osversion.find('.')

        if osversion.find(' ')>0:
            idx2 = osversion.find(' ')
        elif osversion.find('-')>0:
            idx2 = osversion.find('-')
        else:
            idx2 = len(osversion)
        
        new_osv = osversion[0:idx1] + '.' + osversion[idx1+1:idx2].replace('.','')

        try:
            return float(new_osv)
        except:
            return 0

    try:
        return float(osversion)
    except:
        return 0

In [14]:
train['osv'] = train['osv'].apply(osv_processing)
test['osv'] = test['osv'].apply(osv_processing)

In [15]:
train['osv']

0         9.00
1         8.10
2         8.10
3         8.10
4         8.00
          ... 
499995    7.12
499996    8.10
499997    6.01
499998    5.11
499999    8.10
Name: osv, Length: 500000, dtype: float64

In [16]:
# 3.1.3 处理第三个object类型的特征：version  app版本
train['version'].value_counts().index

Index(['0', '5', '8', '4', '3', '7', '2', '1', '11', '6', 'V3', '9', 'v1',
       '10', 'P_Final_6', 'V6', 'GA3', 'GA2', '15', 'V2', ' 2', '50'],
      dtype='object')

In [17]:
# 由上结果，结合生活经验
# V3 可能是某个app的第3个版本
# v1 可能是某个app的第1个版本
# ...
def version_processing(v):
    if v=='V3':
        return 3
    if v=='v1':
        return 1
    if v=='P_Final_6':
        return 6
    if v=='V6':
        return 6
    if v=='GA3':
        return 3
    if v=='GA2':
        return 2
    if v=='V2':
        return 2
    if v=='50':
        return 5

    return int(v)

In [18]:
train['version'] = train['version'].apply(version_processing)
test['version'] = test['version'].apply(version_processing)

train['version'] = train['version'].astype('int')
test['version'] = test['version'].astype('int')

In [19]:
# 3.1.4 处理第四个object类型的特征：fea_hash  用户特征编码（具体物理含义略去）
# 还有个fea1_hash，也是用户特征编码（具体物理含义略去） 这两个一起处理
train['fea_hash'].value_counts()

68083895      110
235856055      99
51306679       91
2815114810     77
16777343       60
             ... 
1826339879      1
1284649018      1
1897970805      1
641195487       1
4103514747      1
Name: fea_hash, Length: 402980, dtype: int64

In [20]:
# 用户特征编码，具体含义不明，直接按照长度进行处理，默认超过16的意义不大
# 其实也是增加了新的特征
train['fea_hash_len'] = train['fea_hash'].map(lambda x: len(str(x)))
train['fea1_hash_len'] = train['fea1_hash'].map(lambda x: len(str(x)))
train['fea_hash'] = train['fea_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))
train['fea1_hash'] = train['fea1_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))

test['fea_hash_len'] = test['fea_hash'].map(lambda x: len(str(x)))
test['fea1_hash_len'] = test['fea1_hash'].map(lambda x: len(str(x)))
test['fea_hash'] = test['fea_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))
test['fea1_hash'] = test['fea1_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))

## Step3.2: 增加新的特征

In [21]:
# 3.2.1 构造时间相关特征
train['timestamp'].value_counts()
train['timestamp'].apply(lambda x: datetime.fromtimestamp(x/1000))

0        2019-06-07 15:32:08.241721
1        2019-06-08 19:40:20.922229
2        2019-06-06 23:58:17.549109
3        2019-06-09 08:59:37.719685
4        2019-06-07 08:28:21.478553
                    ...            
499995   2019-06-06 23:14:18.267242
499996   2019-06-06 17:40:54.045280
499997   2019-06-05 03:14:57.290064
499998   2019-06-07 00:59:27.651521
499999   2019-06-07 23:32:46.472229
Name: timestamp, Length: 500000, dtype: datetime64[ns]

In [22]:
train['timestamp'] = train['timestamp'].apply(lambda x: datetime.fromtimestamp(x/1000))
test['timestamp'] = test['timestamp'].apply(lambda x: datetime.fromtimestamp(x/1000))

In [23]:
# 提取时间多尺度 训练集
time = pd.DatetimeIndex(train['timestamp'])
train['year'] = time.year
train['month'] = time.month
train['day'] = time.day
train['week_day'] = time.weekday 
train['hour'] = time.hour
train['minute'] = time.minute

start_time = train['timestamp'].min()
train['diff_time'] = train['timestamp'] - start_time  # 时间差
train['diff_time'] = train['diff_time'].dt.days + train['diff_time'].dt.seconds/3600/24

train[['timestamp', 'year', 'month', 'day', 'week_day', 'hour', 'minute', 'diff_time']]

,timestamp,year,month,day,week_day,hour,minute,diff_time
0,2019-06-07 15:32:08.241721,2019,6,7,4,15,32,4.647303
1,2019-06-08 19:40:20.922229,2019,6,8,5,19,40,5.819664
2,2019-06-06 23:58:17.549109,2019,6,6,3,23,58,3.998796
3,2019-06-09 08:59:37.719685,2019,6,9,6,8,59,6.374722
4,2019-06-07 08:28:21.478553,2019,6,7,4,8,28,4.353009
...,...,...,...,...,...,...,...,...
499995,2019-06-06 23:14:18.267242,2019,6,6,3,23,14,3.968252
499996,2019-06-06 17:40:54.045280,2019,6,6,3,17,40,3.736713
499997,2019-06-05 03:14:57.290064,2019,6,5,2,3,14,2.135370
499998,2019-06-07 00:59:27.651521,2019,6,7,4,0,59,4.041273


In [24]:
# 提取时间多尺度 测试集
time = pd.DatetimeIndex(test['timestamp'])
test['year'] = time.year
test['month'] = time.month
test['day'] = time.day
test['week_day'] = time.weekday 
test['hour'] = time.hour
test['minute'] = time.minute

test['diff_time'] = test['timestamp'] - start_time  # 时间差
test['diff_time'] = test['diff_time'].dt.days + test['diff_time'].dt.seconds/3600/24

test[['timestamp', 'year', 'month', 'day', 'week_day', 'hour', 'minute', 'diff_time']]

,timestamp,year,month,day,week_day,hour,minute,diff_time
0,2019-06-07 09:43:20.477148,2019,6,7,4,9,43,4.405081
1,2019-06-05 20:53:33.449314,2019,6,5,2,20,53,2.870509
2,2019-06-04 10:08:24.732394,2019,6,4,1,10,8,1.422488
3,2019-06-05 01:03:10.643892,2019,6,5,2,1,3,2.043854
4,2019-06-05 08:16:34.450110,2019,6,5,2,8,16,2.344826
...,...,...,...,...,...,...,...,...
149995,2019-06-08 09:21:33.629823,2019,6,8,5,9,21,5.389954
149996,2019-06-07 07:15:57.431662,2019,6,7,4,7,15,4.302731
149997,2019-06-09 08:48:59.661749,2019,6,9,6,8,48,6.367338
149998,2019-06-05 06:38:14.527250,2019,6,5,2,6,38,2.276539


In [25]:
# 3.2.2 构造dev相关特征
train['dev_ratio'] = train['dev_height'] / train['dev_width']
test['dev_ratio'] = test['dev_height'] / test['dev_width']
train['dev_area'] = train['dev_height'] * train['dev_width']
test['dev_area'] = test['dev_height'] * test['dev_width']
train['dev_area'].value_counts()

0.0           107014
2073600.0      70352
273600.0       38402
921600.0       32560
2527200.0      30869
               ...  
198400.0           1
25369344.0         1
4731000.0          1
9462912.0          1
1048320.0          1
Name: dev_area, Length: 1054, dtype: int64

In [26]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 31 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   android_id     500000 non-null  int64         
 1   apptype        500000 non-null  int64         
 2   carrier        500000 non-null  float64       
 3   dev_height     500000 non-null  float64       
 4   dev_ppi        500000 non-null  float64       
 5   dev_width      500000 non-null  float64       
 6   label          500000 non-null  int64         
 7   lan            500000 non-null  float64       
 8   media_id       500000 non-null  int64         
 9   ntt            500000 non-null  float64       
 10  os             500000 non-null  object        
 11  osv            500000 non-null  float64       
 12  package        500000 non-null  int64         
 13  sid            500000 non-null  int64         
 14  timestamp      500000 non-null  datetime64[ns]
 15  

In [27]:
# 3.2.3 构造app版本与操作系统版本
train['osv_version'] = train['osv'] - train['version']
test['osv_version'] = test['osv'] - test['version']
test['osv_version'].value_counts()

 8.10    29881
 9.00    15498
 5.10     8231
 4.00     7910
 6.01     6636
         ...  
 7.31        1
-2.70        1
-2.80        1
 5.30        1
 3.20        1
Name: osv_version, Length: 206, dtype: int64

# Step4: 建模 训练 预测

In [28]:
import xgboost as xgb
# 建模
xgb = xgb.XGBClassifier(
            max_depth=6, learning_rate=0.05, n_estimators=6000, 
            objective='binary:logistic',  # tree_method='gpu_hist',
            subsample=0.8, colsample_bytree=0.8,
            min_child_samples=3, eval_metric='auc', reg_lambda=0.5
        )

In [29]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 32 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   android_id     500000 non-null  int64         
 1   apptype        500000 non-null  int64         
 2   carrier        500000 non-null  float64       
 3   dev_height     500000 non-null  float64       
 4   dev_ppi        500000 non-null  float64       
 5   dev_width      500000 non-null  float64       
 6   label          500000 non-null  int64         
 7   lan            500000 non-null  float64       
 8   media_id       500000 non-null  int64         
 9   ntt            500000 non-null  float64       
 10  os             500000 non-null  object        
 11  osv            500000 non-null  float64       
 12  package        500000 non-null  int64         
 13  sid            500000 non-null  int64         
 14  timestamp      500000 non-null  datetime64[ns]
 15  

In [30]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 31 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   android_id     150000 non-null  int64         
 1   apptype        150000 non-null  int64         
 2   carrier        150000 non-null  float64       
 3   dev_height     150000 non-null  float64       
 4   dev_ppi        150000 non-null  float64       
 5   dev_width      150000 non-null  float64       
 6   lan            150000 non-null  float64       
 7   media_id       150000 non-null  int64         
 8   ntt            150000 non-null  float64       
 9   os             150000 non-null  object        
 10  osv            150000 non-null  float64       
 11  package        150000 non-null  int64         
 12  sid            150000 non-null  int64         
 13  timestamp      150000 non-null  datetime64[ns]
 14  version        150000 non-null  int64         
 15  

In [31]:
%%time
# 模型训练
xgb.fit(train.drop(['os', 'sid', 'timestamp', 'label'], axis=1), train['label'])
result = xgb.predict(test.drop(['os', 'sid', 'timestamp'], axis=1))
result

[19:20:20] WARNING: ../src/learner.cc:541: 
Parameters: { min_child_samples } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


CPU times: user 1min 6s, sys: 4.06 s, total: 1min 10s
Wall time: 44.8 s


array([0, 1, 0, ..., 1, 1, 1])

In [33]:
ressult_final = pd.DataFrame(test['sid'])
ressult_final['label'] = result
ressult_final.to_csv('./result.csv', index=False)
ressult_final

,sid,label
0,1440682,0
1,1606824,1
2,1774642,0
3,1742535,0
4,1689686,1
...,...,...
149995,1165373,1
149996,1444115,1
149997,1134378,1
149998,1700238,1
